# Xanadu Challenge | Qiskit Fall Fest Latino 2023

## Presented by: QuantumForce

### Members:

*   Danesh Morales Hashemi (bobbybomb#3873)
*   Alejandro Sebastián Benítez Rodríguez (sebas_ben135#8087)
*   José Felipe De León (josdelen#4499)
*   David Rivera Cardenas (david.rivera#7896)
*   Marc Edson Urcia Soto (marcksito#8235)

For this challenge we provided a variational quantum circuit in PennyLane that depends on a set of trainable parameters. The circuit produces a single number as the expectation value of a fixed measurement.

The minimum expectation value of this circuit can be produced by optimizing its parameters. This leads to converting the circuit into a QNode. It was coded using one of the PennyLane optimizers called AdamOptimizer.

First, we will start by importing the libraries we are going to need and declare certain constants

In [14]:
import pennylane as qml
from pennylane import numpy as np

WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

Now, we will define the function `variational_circuit` which was already given to us.

In [3]:
def variational_circuit(params,hamiltonian):
    """
    This is a template variational quantum circuit containing a fixed layout of gates with variable
    parameters. To be used as a QNode, it must either be wrapped with the @qml.qnode decorator or
    converted using the qml.QNode function.

    The output of this circuit is the expectation value of a Hamiltonian, somehow encoded in
    the hamiltonian argument

    Args:
        - params (np.ndarray): An array of optimizable parameters of shape (30,)
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value is returned.

    Returns:
        (float): The expectation value of the Hamiltonian
    """
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))

We need to create a function `optimize_circuit` so that it minimizes the variational circuit. We were given a template for the function that we had to fill up, the template is the following:

In [5]:
def optimize_circuit(hamiltonian):
    """Minimize the variational circuit and return its minimum value.
    You should create a device and convert the variational_circuit function
    into an executable QNode.
    Next, you should minimize the variational circuit using gradient-based
    optimization to update the input params.
    Return the optimized value of the QNode as a single floating-point number.

    Args:
        - params (np.ndarray): Input parameters to be optimized, of dimension 30
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value you should minimize.
    Returns:
        float: the value of the optimized QNode
    """

    hamiltonian = np.array(hamiltonian, requires_grad = False)

    hamiltonian = np.array(hamiltonian,float).reshape((2 ** WIRES), (2 ** WIRES))

    ### WRITE YOUR CODE BELOW THIS LINE

    ### Solution Template

    dev = None # Initialize the device.

    circuit = None # Instantiate the QNode from variational_circuit.

    # Write your code to minimize the circuit

    return None # Return the value of the minimized QNode

We will now explain how we filled up the function template. Since we are given `hamiltonian`, we need `params` to call the function `variational circuit`. `params` will be an array of initial values of same size as `hamiltonian` that will be updated by the optimizer. Deciding which initial values `params` has is very crucial to find the most accurate optimization. We tried with different options, such as an array of zeroes (given by `np.zeros()`) or just an array of random numbers. After many different attempts, the best option was an array of uniformly distributed values. By initializing parameters within $[0, 2π]$, we are providing a reasonable starting point for the optimization algorithm to explore the parameter space effectively.

In [6]:
params = np.random.uniform(0, 2*np.pi, size=(NUM_PARAMETERS))

To initialize the device and circuit we use as a reference the articles [Variational classifier](https://pennylane.ai/qml/demos/tutorial_variational_classifier/) and [qml.QNode documentation](https://docs.pennylane.ai/en/stable/code/api/pennylane.QNode.html) from the PennyLane website.



In [7]:
dev = qml.device("default.qubit", wires = WIRES)

circuit = qml.QNode(variational_circuit, dev)

Now, we need to cost function that finds the error between the current state of your system (in this case, the variational quantum circuit) and the desired outcome (the expectation value of the Hamiltonian). From the article [Basic tutorial: qubit rotation](https://pennylane.ai/qml/demos/tutorial_qubit_rotation/) in the PennyLane website we found an example of a cost function that gets the job done.

In [8]:
def cost(parameters):
    return circuit(parameters, hamiltonian)

Choosing an optimizer is an important decision in the optimization process, as different optimizers work differently. PennyLane has [many options for optimizers](https://openqaoa.entropicalabs.com/optimizers/pennylane-optimizers/). After testing several options, the [Adam Optimizer](https://docs.pennylane.ai/en/stable/code/api/pennylane.AdamOptimizer.html) gave us the most accurate results.

In [9]:
optimizer = qml.AdamOptimizer(stepsize=0.1)

It is time to update the values of `params` to minimize the cost function. We can update `params` in a loop which will make the cost function be reduced in every iteration.

In [ ]:
for i in range(100):
    params = optimizer.step(cost, params)

After optimizing `params`, our desired output is the minimum expectation value of `hamiltonian`, given by `circuit(params,hamiltonian)`. We now provide the complete code of the `optimize_circuit` function.

In [32]:
def optimize_circuit(hamiltonian):
    """Minimize the variational circuit and return its minimum value.
    You should create a device and convert the variational_circuit function
    into an executable QNode.
    Next, you should minimize the variational circuit using gradient-based
    optimization to update the input params.
    Return the optimized value of the QNode as a single floating-point number.

    Args:
        - params (np.ndarray): Input parameters to be optimized, of dimension 30
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value you should minimize.
    Returns:
        float: the value of the optimized QNode
    """

    hamiltonian = np.array(hamiltonian, requires_grad = False)

    hamiltonian = np.array(hamiltonian,float).reshape((2 ** WIRES), (2 ** WIRES))

    ### WRITE YOUR CODE BELOW THIS LINE

    ### Solution Template

    params = np.random.uniform(0, 2*np.pi, size=(NUM_PARAMETERS))

    dev = qml.device("default.qubit", wires = WIRES)

    circuit = qml.QNode(variational_circuit, dev)

    # Write your code to minimize the circuit

    def cost(parameters):
        return circuit(parameters, hamiltonian)

    optimizer = qml.AdamOptimizer(stepsize=0.1)


    for i in range(100):
        params = optimizer.step(cost, params)

    min_expectation = circuit(params, hamiltonian)

    return min_expectation.item()

We provide a function that checks if our output matches the desired output, with a tolerance of 0.001, as instructed.

In [19]:
def check_result(hamiltonian,desired_output):

    tolerance = 0.001

    return True if np.abs(optimize_circuit(hamiltonian) - desired_output) < tolerance else False

We will now test the two test cases given.

In [20]:
hamiltonian_1 = [0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154, 0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282, 0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205, 0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]
output_1 = 0.61745341

hamiltonian_2 = [0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206]
output_2 = 0.00246488

In [33]:
check_result(hamiltonian_1, output_1)

True

In [34]:
check_result(hamiltonian_2, output_2)

True

## Conclusion

* This work provides a practical demonstration of variational quantum circuit optimization using PennyLane.

* The final set of optimized parameters represents a solution that minimizes the Hamiltonian's expectation value.

* Using the Adam Optimizer, we iteratively adjusted the circuit parameters, achieving convergence within 200 iterations.
